# 8. Try to build your own Canny edge detector (not the one that is called directly from OpenCV) and detect the edges in the image.  

## Canny edge detector (from scratch)

This section implements the full Canny pipeline without calling OpenCV's `Canny`:

1) Smooth image with a Gaussian to reduce noise
2) Compute x/y derivatives (Sobel)
3) Gradient magnitude and orientation
4) Non-maximum suppression to thin edges
5) Double thresholding (low/high)
6) Edge tracking by hysteresis (linking weak to strong)

We'll provide a reusable function and a small demo. You can either set `image_path` to a local file, or it will generate a synthetic test image.

In [ ]:
# Imports and small utilities
import numpy as np
import math
import matplotlib.pyplot as plt

def to_grayscale(img):
    """Convert RGB or RGBA image to grayscale float32 [0,1]."""
    img = np.asarray(img)
    if img.ndim == 2:
        g = img.astype(np.float32)
        # Normalize if likely 0..255
        if g.max() > 1.5:
            g = g / 255.0
        return g
    if img.shape[-1] == 4:  # drop alpha
        img = img[..., :3]
    # BT.709 luma weights
    w = np.array([0.2126, 0.7152, 0.0722], dtype=np.float32)
    g = np.tensordot(img.astype(np.float32), w, axes=([-1],[0]))
    if g.max() > 1.5:
        g = g / 255.0
    return g

def imshow_grid(images, titles=None, cmap='gray', ncols=3, figsize=(12, 8)):
    images = list(images)
    n = len(images)
    ncols = min(ncols, n) if n>0 else 1
    nrows = (n + ncols - 1) // ncols
    plt.figure(figsize=figsize)
    for i, img in enumerate(images):
        ax = plt.subplot(nrows, ncols, i+1)
        if img.ndim == 2:
            ax.imshow(img, cmap=cmap)
        else:
            ax.imshow(img)
        if titles and i < len(titles):
            ax.set_title(titles[i])
        ax.axis('off')
    plt.tight_layout()

In [ ]:
# Pre-cleaning utilities (grayscale, contrast stretch, optional median filter)
import numpy as np

def median_filter2d(img, ksize=3, padding='reflect'):
    assert ksize % 2 == 1, 'ksize must be odd'
    k = ksize // 2
    if padding == 'reflect':
        img_p = np.pad(img, ((k,k),(k,k)), mode='reflect')
    elif padding == 'edge':
        img_p = np.pad(img, ((k,k),(k,k)), mode='edge')
    else:
        img_p = np.pad(img, ((k,k),(k,k)), mode='constant')
    H, W = img.shape
    out = np.empty_like(img, dtype=np.float32)
    for i in range(H):
        for j in range(W):
            window = img_p[i:i+ksize, j:j+ksize]
            out[i, j] = np.median(window)
    return out

def contrast_stretch01(img, low=1, high=99):
    p_low = np.percentile(img, low)
    p_high = np.percentile(img, high)
    if p_high <= p_low:
        return np.clip(img, 0.0, 1.0).astype(np.float32)
    out = (img - p_low) / (p_high - p_low)
    return np.clip(out, 0.0, 1.0).astype(np.float32)

def pre_clean(image, do_contrast=True, clip_limits=(1,99), median_ksize=None):
    """Convert to grayscale [0,1], optional contrast stretch and median denoise."""
    g = to_grayscale(image).astype(np.float32)
    if do_contrast:
        g = contrast_stretch01(g, low=clip_limits[0], high=clip_limits[1])
    if median_ksize and median_ksize >= 3:
        g = median_filter2d(g, ksize=int(median_ksize), padding='reflect')
    return g

In [ ]:
# Common: 2D convolution (single-channel)
import numpy as np

def conv2d(image, kernel, padding='reflect'):
    img = image.astype(np.float32)
    ker = np.flipud(np.fliplr(kernel.astype(np.float32)))  # convolution flip
    kh, kw = ker.shape
    ph, pw = kh//2, kw//2
    if padding == 'reflect':
        img_p = np.pad(img, ((ph, ph), (pw, pw)), mode='reflect')
    elif padding == 'edge':
        img_p = np.pad(img, ((ph, ph), (pw, pw)), mode='edge')
    else:
        img_p = np.pad(img, ((ph, ph), (pw, pw)), mode='constant')
    H, W = img.shape
    out = np.zeros_like(img, dtype=np.float32)
    for i in range(H):
        for j in range(W):
            region = img_p[i:i+kh, j:j+kw]
            out[i, j] = np.sum(region * ker)
    return out

In [ ]:
# Step 0: Gaussian smoothing
import numpy as np

def gaussian_kernel(size=5, sigma=1.4):
    assert size % 2 == 1, 'Kernel size must be odd'
    k = size // 2
    x = np.arange(-k, k+1)
    y = np.arange(-k, k+1)
    X, Y = np.meshgrid(x, y)
    g = np.exp(-(X**2 + Y**2) / (2*sigma**2))
    g /= g.sum()
    return g.astype(np.float32)

def gaussian_blur(img, size=5, sigma=1.4):
    gk = gaussian_kernel(size=size, sigma=sigma)
    return conv2d(img, gk, padding='reflect')

In [ ]:
# Step 1: Compute x and y derivative images (Sobel)
import numpy as np

def sobel_derivatives(img):
    Sx = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]], dtype=np.float32)
    Sy = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1,-2,-1]], dtype=np.float32)
    Ix = conv2d(img, Sx, padding='reflect')
    Iy = conv2d(img, Sy, padding='reflect')
    return Ix, Iy

In [ ]:
# Step 2: Gradient magnitude and orientation
import numpy as np

def gradient_magnitude_orientation(Ix, Iy, use_l2=True):
    if use_l2:
        mag = np.hypot(Ix, Iy).astype(np.float32)
    else:
        mag = (np.abs(Ix) + np.abs(Iy)).astype(np.float32)
    theta = np.degrees(np.arctan2(Iy, Ix)).astype(np.float32)  # -180..180
    return mag, theta